Tarkastellaan asuntojen hintojen muodustumista.

Datassa seuraavat muuttujat 
['Kaupunginosa','Huoneet','Talotiedot','m2','Vh','Neliohinta','Rv']

Millaisia korrelaatiota eri tekijöiden välillä on?
Mitkä tekijät selittävät asunnon velatonta hintaa tai neliöhintaa? 


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [39]:
rawdata='C://Python34/datasets/asunnot_250316_cleaned_numerical2.csv'
df_raw = pd.read_csv(rawdata,header=None)
column_names = ['Kaupunginosa','Huoneet','Talotiedot','m2','Vh','Neliohinta','Rv']
#print(df_raw.head(2))
df=df_raw.ix[:, [0,1,2,3,4,5,6]]
#print(df.head())


In [38]:
print(np.cov(df,rowvar=False))

[[  1.02510241e+01   1.96432279e-01   6.55592261e-02   1.24690533e+01
   -3.52271026e+03  -3.33294082e+02  -4.91624351e+00]
 [  1.96432279e-01   8.99178858e-01   3.34478528e-01   3.17630996e+01
    5.77472821e+04  -1.35255158e+02   3.72024540e-01]
 [  6.55592261e-02   3.34478528e-01   7.07541293e-01   1.16292345e+01
    2.33663896e+04  -8.37534686e+00   5.37989618e-02]
 [  1.24690533e+01   3.17630996e+01   1.16292345e+01   1.80333955e+03
    2.76126320e+06  -9.35927400e+03  -5.74516527e+01]
 [ -3.52271026e+03   5.77472821e+04   2.33663896e+04   2.76126320e+06
    7.62388314e+09   1.97673039e+07   3.75706475e+05]
 [ -3.33294082e+02  -1.35255158e+02  -8.37534686e+00  -9.35927400e+03
    1.97673039e+07   5.27443598e+05   4.85604040e+03]
 [ -4.91624351e+00   3.72024540e-01   5.37989618e-02  -5.74516527e+01
    3.75706475e+05   4.85604040e+03   3.21817197e+02]]


Mitkä tekijät korreloivat eniten keskenään? Kovarianssimatriisista:
Huoneet vs. Vh 10+4, 
Talotiedot vs. Vh 10+4, 
m2 vs. Vh 10+6, 
Vh vs. neliohinta 10+5, 
Vh vs. Rv 10+3, 
Rv vs. Neliohinta 10+3   

In [28]:
from sklearn.decomposition import PCA
pca = PCA()

In [34]:
df_pca = pca.fit_transform(df)
#print(np.cov(df_pca, rowvar=False))

In [35]:
print(pca.explained_variance_)
#plt.plot(pca.explained_variance_);
#plt.show()

[  5.25870303e+05   2.76244912e+02   1.11742888e+00   4.40149456e-01]
